In [1]:
!python --version

Python 3.10.15


In [2]:
!pip install -r requirements.txt

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.7-py3-none-any.whl.metadata (2.9 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached unstructured-0.16.5-py3-none-any.whl.metadata (24 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached unstructured_inference-0.8.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached pillow_heif-0.20.0-cp310-cp310-win_amd64.whl.metadata (10.0 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_ollama-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp

In [3]:
import os

from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM

In [4]:
load_dotenv('.env')

OLLAMA_HOST = os.getenv('OLLAMA_HOST')
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL')

In [5]:
llm = OllamaLLM(
    base_url=OLLAMA_HOST,
    model=OLLAMA_MODEL,
    verbose=True
)

In [6]:
md_directory = "data/"
md_files = [f for f in os.listdir(md_directory) if f.endswith('.md')]

print(f"Markdown files: {md_files}")

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)

all_pages = []
for md_file in md_files:
    md_path = os.path.join(md_directory, md_file)
    loader = UnstructuredMarkdownLoader(md_path)
    pages = loader.load_and_split(text_splitter)
    all_pages.extend(pages)

    print(f"Content of {md_file}:")
    for page in pages:
        print(page)

    all_pages.extend(pages)
    print(f"Loaded {len(pages)} pages from {md_file}")

print(f"Total number of pages loaded: {len(all_pages)}")

Markdown files: ['Лекция 1. Алгебра матриц.md']
Content of Лекция 1. Алгебра матриц.md:
page_content='\title{ Лекция 9. Алгебра матриц }

\section*{Матрица и ее частные виды}

Прямоугольной матрицей размера $m \times n$ называется совокупность $m \cdot n$ чисел, расположенных в виде прямоугольной таблицы, содержащей $m$ строк и $n$ столбцов.

Мы будем записывать матрицу в виде $$ A=\left(\begin{array}{cccc} a_{11} & a_{12} & \ldots & a_{1 n} \ a_{21} & a_{22} & \ldots & a_{2 n} \ \vdots & \vdots & : & : \ a_{m 1} & a_{m 2} & \ldots & a_{m n} \end{array}\right)=\left(a_{i j}\right)_{m \times n} $$

Числа $a_{i j}$ называются элементами матрицы, первый индекс матрицы (i) обозначает номер строки, второй $(j)$ - номер столбца. $$ \begin{aligned} & A=\left(\begin{array}{ccc} j & 1 & -3 \ 4 & -j & 5 \end{array}\right) \text {-комплексная матрица размера } 2 \times 3 . \ & B=\left(\begin{array}{ll} 1 & 2 \ 3 & 4 \end{array}\right) \text { - вещественная матрица размера } 2 \times 2 . \end{ali

In [7]:
embedding_model_nomic = OllamaEmbeddings(model="nomic-embed-text:latest")
db = Chroma.from_documents(all_pages, embedding_model_nomic)
retriever = db.as_retriever(search_kwargs={"k": 3})

In [8]:
template = """
Вы - преподаватель математики с искусственным интеллектом.
Ваша задача - создать корректные задания по математике на основе контекста.

### Контекст:
{context}

### Вопрос:
{input}

### Ответ:
"""

In [9]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [10]:
def get_response(query: str) -> str:
    """
    Функция, возвращающая ответ на вопрос, основанный на текстовых данных.

    :param query: текст вопроса
    :return: текст ответа
    """
    response = retrieval_chain.invoke({"input": query})
    return response["answer"]


In [12]:
query = input()
response = get_response(query)
print(response)

 Составь практическое задание, следуя следующим инструкциям: 1. Сгенерируй матрицу 2x2, где каждая ячейка содержит целое или десятичное число, записанное в форме, допускающей ввод с клавиатуры (целое число или десятичная дробь). 2. После матрицы напиши вопрос: "Дана матрица размером 2 на 2. Чему равен определитель этой матрицы?". 3. Предложи 4 варианта ответа, где только один из них верен. Варианты ответа должны быть написаны в следующем формате: а), б), в) и так далее, за исключением букв Ё, З, Й, О, Ч, Ь, Ы, Ъ. Варианты ответа должны включать целое число или десятичную дробь в формате: например, 13, -5, 2.3, 0.15 и т. д. При этом десятичная дробь должна быть конечной, т. е. ответ в 0,1 (эквивалентно 1/10) допускается, а от 0,07142857142857142857142857142857 (эквивалентно 1/14) не допускается.


: 

1. С Generationruем матрицу \(2 \times 2\), где каждая cellua contains an integer or a decimal fraction in the form, allowing input from keyboard (integer number or decimal fraction). 
2. After generating the matrix, write the question: "What is the determinant of this matrix?".
3. Provide four possible answers, where only one of them is correct. Answers should be written in the following format: 
   \[
   \text{а)}, \text{б)}, \text{в)} \text{и так далоelves, за исключением букв Ё, З, Й, О, Ч, Ь, Ы, Ъ.},
   \]
4. Possible answers should include whole numbers or decimal fractions in the form: 
   \[
   \text{"13", -5, 2.3, 0.15 and so on, where the fraction can be equivalent to 1/10, but not as 0.07142857142857142857142857142857}.
   \]

#### Step-by-step solution:

Let's consider an example of generating a matrix and calculating its determinant.

**Step 1:** Generate a \(2 \times 2\) matrix with given entries. Suppose the matrix is:
\[
A = \begin{pmatrix}
a & b \\
c & d
\end{pmatr